In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master Thesis
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Master Thesis
candidate_sentences.csv               data/                            PreProcessing.ipynb
candidate_sentences_pred_CLUSTER.csv  dataAnalysis.ipynb               sentimentAggregation.ipynb
candidate_sentences_pred.csv          FindCandidates_V2_onlyESG.ipynb  SP500.ipynb
CB_sentence/                          full_gov/
ClassificationPipeline_AD.ipynb       full_soc/


In [ ]:
import pandas as pd
import numpy as np
import os

path = './data/ESG'
bbgNames = os.listdir('./data/Results')

Get Refinitiv ESG Scores

In [ ]:
scoresRefiv = {}
xls = pd.ExcelFile(path + '/RefinitivESG.xlsx')

for s in xls.sheet_names:
  scoresRefiv[s] = xls.parse(s).iloc[:,1:]

names = scoresRefiv['2021']['Company Common Name'].tolist()
mismatchedNames = []
sameNames = pd.DataFrame(columns=['Refinitiv','BBG'])
cols = scoresRefiv['2021'].columns[1:]
comps = {}
for n in names:
  df = pd.DataFrame(index = ['2021','2020','2019','2018','2017','2016'], columns = cols)
  for k,v in scoresRefiv.items():
    try:
      df.loc[[k],:] = v[v['Company Common Name']==n].iloc[:,1:].values[0]
    except:
      continue
  comps[n] = df

  if n.upper() in bbgNames:
    #sameNames = sameNames.append(pd.DataFrame([n,n.upper()]).T.rename(columns = {0:'Refinitiv',1:'BBG'}))
    sameNames = pd.concat([sameNames,pd.DataFrame([n,n.upper()]).T.rename(columns = {0:'Refinitiv',1:'BBG'})])
  else:
    mismatchedNames.append(n)

  '''
  try:
    df.to_csv('./data/Results/%s/esgRefinitiv.csv'%(n.upper()))
  except:
    mismatchedNames.append(n)
  '''

Some names from Refinitiv mismatch with Bloomberg. I manually verify and re-match to save the Refinitiv data in the right folders (only need to run once)

In [ ]:
mismatchedNames = pd.read_csv('./data/ESG/names.csv').dropna()
mismatchedNamesDict = mismatchedNames.set_index('Refinitiv').to_dict()['BBG']

for k,v in mismatchedNamesDict.items():
  ref = k
  bbg = v
  comps[k].to_csv('./data/Results/%s/esgRefinitiv.csv'%v)

#RepRisk data to folders
---

Get RepRisk Company name equivalent in BBG

In [ ]:
allCompNames = pd.read_csv('./data/ESG/names.csv').dropna()
allCompNames = pd.concat([allCompNames,sameNames])
allCompNames.sort_values(by=['BBG'], inplace=True)
allCompNames['isin'] = [0]*len(allCompNames)

isin = pd.read_excel('./data/ESG/sp500_isin.xlsx').drop(columns=['Unnamed: 0']).set_index('Company Common Name')
isinDict = isin.to_dict()['ISIN']

nope = []
for i in range(len(allCompNames)):
  name = allCompNames.iloc[i,0]
  try:
    allCompNames.iloc[i,2] = isinDict[name]
  except:
    nope.append(name)

#allCompNames.set_index('isin',inplace=True)
allCompNames

Add reprisk names by isin (run once)
currently, the only manual changes made were to add Advance Auto Parts, Alphabet Inc, Fox Corp, News Corp, DXC Technologies, Linde PLC, Newell Brands . All others do not exist in RepRisk

In [ ]:
repNames = pd.read_csv('./data/ESG/2022.csv')[['company_name','primary_isin']].set_index('primary_isin')
repNamesU = repNames.drop_duplicates()
allCompNames['RepRisk'] = [np.nan]*len(allCompNames)
nameDict = repNamesU.to_dict()['company_name']

nopeRep = []
for i in range(len(allCompNames)):
  isin = allCompNames.index[i]
  try:
    allCompNames.iloc[i,2] = nameDict[isin]
  except:
    nopeRep.append(isin)
#allCompNames[allCompNames.isna().any(axis=1)] #get names of companies that were not found in RepRisk --> manually check

allCompNames.to_csv('./data/ESG/allNamesFromProviders.csv')

Save RepRisk Scores to corresponding folders


In [ ]:
def saveScoresToCompByYears(yearsDf,namesDict,sortby='date',toSave=False):
  comps = {}
  for m,n in namesDict.items():
    print('--- Starting for %s ---'%m)

    comps[m] = yearsDf[yearsDf['company_name']==n].sort_values(sortby).reset_index().drop(columns=['index','company_name','primary_isin'])
    comps[m]['current_rriDiff'] = comps[m]['current_rri'].diff()

    if toSave:
      comps[m].iloc[1:,:].to_csv('./data/Results/%s/RepRiskRRI.csv'%m)

    print('--- Done for %s ---'%m)

  return comps

In [ ]:
def saveScoresToCompByYears(yearsDf,namesDict,valuesDict=None,sortby='date',toSave=False):
  comps = {}
  for m,n in namesDict.items():
    print('--- Starting for %s ---'%m)

    comps[m] = yearsDf[yearsDf['company_name']==n].sort_values(sortby).reset_index().drop(columns=['index','company_name','primary_isin'])

    df = comps[m].copy()

    if valuesDict == None:
      comps[m]['current_rriDiff'] = comps[m]['current_rri'].diff()
      if toSave:
        comps[m].iloc[1:,:].to_csv('./data/Results/%s/RepRiskRRI.csv'%m)

    else:
      c = []
      for r in df['reprisk_rating'].values.tolist():
        c.append(valuesDict[r])

      df['rating'] = c
      df['ratingchange'] = df['rating'].diff()
      if toSave:
        df.iloc[1:,:].to_csv('./data/Results/%s/RepRiskRating.csv'%m)

    print('--- Done for %s ---'%m)

  #return comps

Give numerical values to RepRisk Ratings

In [ ]:
years = list(range(2015,2023))
credit_ratings = {
    "AAA": 10,
    "AA": 9,
    "A": 8,
    "BBB": 7,
    "BB": 6,
    "B": 5,
    "CCC": 4,
    "CC": 3,
    "C": 2,
    "D": 1
}

cnt=0
for y in years:
  if cnt==0:
    scoresRRI = pd.read_csv(path + '/RepRisk/%s.csv'%y)
  else:
    scoresRRI = pd.concat([scoresRRI,pd.read_csv(path + '/RepRisk/%s.csv'%y)])
  cnt+=1

namesDict = pd.read_csv(path + '/allNamesFromProviders.csv')[['BBG','RepRisk']].dropna().set_index('BBG').to_dict()['RepRisk']
saveScoresToCompByYears(scoresRRI,namesDict,None, sortby='date',toSave=False)
saveScoresToCompByYears(scoresRRI,namesDict,credit_ratings, sortby='date',toSave=True)



manually save for Advance Auto Parts Inc. (run once)

In [ ]:
years = list(range(2015,2023))
cnt=0
for y in years:
  if cnt==0:
    scoresRRIAAP = pd.read_csv(path + '/RepRisk/%s_AAP.csv'%y)
  else:
    scoresRRIAAP = pd.concat([scoresRRIAAP,pd.read_csv(path + '/RepRisk/%s_AAP.csv'%y)])
  cnt+=1

scoresRRIAAP = scoresRRIAAP.sort_values('date').reset_index().drop(columns=['index','company_name','primary_isin'])
df = scoresRRIAAP.copy()

scoresRRIAAP['current_rriDiff'] = scoresRRIAAP['current_rri'].diff()
scoresRRIAAP.iloc[1:,:].to_csv('./data/Results/ADVANCE AUTO PARTS INC/RepRiskRRI.csv')


c = []
for r in df['reprisk_rating'].values.tolist():
  c.append(credit_ratings[r])

df['rating'] = c
df['ratingchange'] = df['rating'].diff()
df.iloc[1:,:].to_csv('./data/Results/ADVANCE AUTO PARTS INC/RepRiskRating.csv')

manually save for DXC, Linde and Newell. (run once)

In [ ]:
years = list(range(2015,2023))

cnt=0
for y in years:
  if cnt==0:
    scoresRRI = pd.read_csv(path + '/RepRisk/%s_stragglers.csv'%y)
  else:
    scoresRRI = pd.concat([scoresRRI,pd.read_csv(path + '/RepRisk/%s_stragglers.csv'%y)])
  cnt+=1

namesDict = {'DXC TECHNOLOGY CO':'DXC Technology Co (formerly Everett SpinCo Inc)', 'LINDE PLC':'Linde PLC','NEWELL BRANDS INC':'Newell Brands Inc (formerly Newell Rubbermaid Inc)'}
saveScoresToCompByYears(scoresRRI,namesDict,credit_ratings, sortby='date',toSave=True)
saveScoresToCompByYears(scoresRRI,namesDict,None, sortby='date',toSave=True)


#Save Bloomberg ESG Scores to corresponding folders + get score changes
---

In [ ]:
scoretemp = pd.read_csv(path + '/SP500_ESGScores.csv', index_col=0).iloc[2:,:]
df = pd.read_csv(path + '/SP500_Names.csv')
scoretemp.columns = scoretemp.iloc[0,:]
sp500 = scoretemp.iloc[3:,:]
sp500 = sp500.astype('float')

temp = pd.DataFrame(np.array(df.columns).reshape((1,2)), columns = ['Ticker', 'CompanyName'])
df.columns = ['Ticker', 'CompanyName']
sp500Names = temp.append(df)

for c in sp500Names.CompanyName:
#  if os.path.exists("./data/Results/%s"%c)==False:
#    os.mkdir("./data/Results/%s"%c)

  ticker = sp500Names[sp500Names['CompanyName']=='%s'%c].Ticker.tolist()[0]

  tickerLoc = sp500.columns.get_loc('%s'%ticker)
  esg = sp500.iloc[:,tickerLoc : tickerLoc+4].dropna(how='all')
  esg.columns = ['Social', 'Gov', 'Env', 'ESG']
  esg[['SocialDiff', 'GovDiff', 'EnvDiff', 'ESGDiff']] = esg.diff()

  esg.iloc[1:,:].to_csv("./data/Results/%s/esg.csv"%c)

  print('Done with %s'%c)

Save Classification to respective companies

In [ ]:
esgPath = './ESG'
rrClasses = pd.read_csv(esgPath+'/RepRisk/repriskClass.csv')
namesDict = pd.read_csv(esgPath + '/allNamesFromProviders.csv')[['BBG','RepRisk']].dropna().set_index('BBG').to_dict()['RepRisk']
uniqueComps = rrClasses.company_name.unique().tolist()
noRiskIncidentsComps = []


for m,n in tqdm(namesDict.items()):
  print('--- Starting for %s ---'%m)

  if n not in uniqueComps:
    noRiskIncidentsComps.append(m)
    continue

  df = rrClasses[rrClasses['company_name']==n].sort_values('incident_date').reset_index().drop(columns=['reprisk_id','index','company_name','primary_isin'])
  df.to_csv('./Results/%s/repriskClasses.csv'%m)

  print('--- Done for %s ---'%m)